In [1]:
from ipynb.fs.full.File_Variables import *

In [2]:
def get_the_WB_runtimes_df(RTVER_NO):
    import math
    import pandas as pd
    from datetime import datetime, timedelta, date

    # data = pd.read_csv("C:/Users/MBhutada.int/OneDrive - Chicago Transit Authority/Desktop/CTA_Mithilesh/CTA Assignments/Metra GTFS for Cubs/Excel and CSV files/Q-04 80 runtimes WK SA SU.csv")
    data = pd.read_csv(csv_name_runtime)
    
    data = data[['RTVER_NO', 'START_PLACE_ID', 'END_PLACE_ID', 'PERIOD_START_TIME', 'PERIOD_END_TIME', 'RVARIANT_ID',  'RUN_TIME']]
    data = data[(data['RVARIANT_ID'] == 'WB') & (data['RTVER_NO'] == RTVER_NO)]

    data['REVISED_PST'] = data['PERIOD_START_TIME'].apply(lambda x: x/36000)
    data['REVISED_PET'] = data['PERIOD_END_TIME'].apply(lambda x: x/36000)

    def convert_time(element):
        subelement = (element - int(str(element)[:2])) * 0.6
        element = float(str(element)[:2]) - 12
        element = element + subelement
        element = "{:.2f}".format(element)
        element = f"{float(element):05.2f}"
        element = str(element)
        if element[3:] == '60':
            element = str(int(element[:2]) + 1) + ":00"
            if len(element) == 5:
                element = element
            else:
                element = "0" + element[0] + ":00"
        return element

    def replace_dot(element):
        element = element.replace('.', ':')
        return element

    def add_seconds_part(element):
        element = element + ":00"
        return element

    data['NEW_PST'] = data['REVISED_PST'].apply(convert_time)
    data['NEW_PET'] = data['REVISED_PET'].apply(convert_time)
    data['NEW_PST'] = data['NEW_PST'].apply(replace_dot)
    data['NEW_PET'] = data['NEW_PET'].apply(replace_dot)
    data['NEW_PST'] = data['NEW_PST'].apply(add_seconds_part)
    data['NEW_PET'] = data['NEW_PET'].apply(add_seconds_part)
    data['REVISED_RUN_TIME'] = data['RUN_TIME'].apply(lambda x: x/600)
    data = data.sort_values(['NEW_PST', 'NEW_PET'])
    data = data.reset_index(drop = True)

    for index, row in data.iterrows():
        if row['NEW_PET'] >= '24:00:00':
            data.loc[index, 'NEW_PET'] = '23:59:59'

    def convert_times(time):
        time_format = "%H:%M:%S"
        time_datetime = datetime.strptime(time, time_format)
        return time_datetime.time()

    x = data[['NEW_PST', 'NEW_PET']].drop_duplicates()
    x = x[(x['NEW_PST'] < '24:00:00') & (x['NEW_PET'] < '24:00:00')]
    x['NEW_PST'] = x['NEW_PST'].apply(convert_times)
    x['NEW_PET'] = x['NEW_PET'].apply(convert_times)

    new_pst = []
    new_pet = []

    x = x.reset_index().drop(['index'], axis = 1).sort_values(by = ['NEW_PST', 'NEW_PET'])

    for i in range(len(x) - 1):
        if (x['NEW_PET'][i] != x['NEW_PST'][i+1]) and (x['NEW_PET'][i] != (datetime.combine(datetime.today(), x['NEW_PST'][i+1]) - timedelta(minutes = 1)).time()) and (x['NEW_PET'][i] < x['NEW_PST'][i+1]):
            new_pst.append(x['NEW_PET'][i])
            new_pet.append(x['NEW_PST'][i+1])

    for i in range(len(new_pet)):
        a = pd.concat([x['NEW_PST'], pd.Series(new_pst)])
        b = pd.concat([x['NEW_PET'], pd.Series(new_pet)])

    a = a.sort_values().reset_index(drop = True)
    b = b.sort_values().reset_index(drop = True)
    df_PST_PET = pd.concat([a, b], axis = 1)
    df_PST_PET = df_PST_PET.rename(columns =  {0 : 'PST', 1: 'PET'})

    comparision_test_dataset = pd.DataFrame(columns = ['Trip', 'Place', 'Time'])
    trip_no = 1
    test_start_time = '03:00:00'
    time_format = "%H:%M:%S"
    test_start_time_datetime = datetime.strptime(test_start_time, time_format)

    if((df_PST_PET['PET'].iloc[-1].hour == 23) and (df_PST_PET['PET'].iloc[-1].minute == 59) and (df_PST_PET['PET'].iloc[-1].second == 59)):
        test_end_time_datetime = datetime(1900, 1, 1, 23, 59, 59)

    else:
        test_end_time_datetime = datetime(1900, 1, 1, df_PST_PET['PET'].iloc[-1].hour, df_PST_PET['PET'].iloc[-1].minute - 1, df_PST_PET['PET'].iloc[-1].second + 59)

    def get_timepoint_runtimes(time):
        data['START-END'] = data['START_PLACE_ID'] + ' - ' + data['END_PLACE_ID']
        return data[(time >= data['NEW_PST']) & (time < data['NEW_PET'])][['START-END', 'REVISED_RUN_TIME']].set_index('START-END').to_dict()

    time = test_start_time

    while test_start_time_datetime <= test_end_time_datetime:
        time_point_runtimes = get_timepoint_runtimes(time)
        time_point_runtime = time_point_runtimes['REVISED_RUN_TIME']

        if len(time_point_runtime) == 0:
            start_time = data[(data['NEW_PET'] <= time)]['NEW_PET'].max()
            end_time = data[(data['NEW_PST'] >= time)]['NEW_PST'].min()
            time_format = "%H:%M:%S"
            start_time_datetime = datetime.strptime(start_time, time_format)
            end_time_datetime = datetime.strptime(end_time, time_format)
            start_time = start_time_datetime
            end_time = end_time_datetime

            dummy_start_time = (start_time - timedelta(minutes = 1)).time()
            dummy_end_time = (end_time + timedelta(minutes = 1)).time()

            p = data[(dummy_start_time.strftime("%H:%M:%S") < data['NEW_PET']) & (dummy_start_time.strftime("%H:%M:%S") >= data['NEW_PST'])]
            n = data[(dummy_end_time.strftime("%H:%M:%S") <= data['NEW_PET']) & (dummy_end_time.strftime("%H:%M:%S") > data['NEW_PST'])]

            timepoint_runtime_dict_previous = {}
            timepoint_runtime_dict_next = {}
            time_point_runtimes_p_ordered = {}
            time_point_runtimes_n_ordered = {}
            time_point_runtimes_p_template = {'IrvBwy - IrvAsh': 0, 'IrvAsh - IrvWes': 0, 'IrvWes - IrvKim': 0, 'IrvKim - IrvBlu': 0, 'IrvBlu - IrvCic': 0, 'IrvCic - IrvCen': 0, 'IrvCen - IrvHar': 0, 'IrvHar - IrvCum': 0}
            time_point_runtimes_n_template = {'IrvBwy - IrvAsh': 0, 'IrvAsh - IrvWes': 0, 'IrvWes - IrvKim': 0, 'IrvKim - IrvBlu': 0, 'IrvBlu - IrvCic': 0, 'IrvCic - IrvCen': 0, 'IrvCen - IrvHar': 0, 'IrvHar - IrvCum': 0}

            for index, row in p.iterrows():
                timepoint_set = row['START-END']
                runtime = row['REVISED_RUN_TIME']
                timepoint_runtime_dict_previous[timepoint_set] = runtime

            for i in time_point_runtimes_p_template:
                time_point_runtimes_p_ordered[i] = timepoint_runtime_dict_previous[i]

            for index, row in n.iterrows():
                timepoint_set = row['START-END']
                runtime = row['REVISED_RUN_TIME']
                timepoint_runtime_dict_next[timepoint_set] = runtime

            for i in time_point_runtimes_n_template:
                time_point_runtimes_n_ordered[i] = timepoint_runtime_dict_next[i]

            time_format = "%H:%M:%S"
            time_for_getting_period_interval = datetime.strptime(time, time_format).time()
            PST_and_PET_for_time = df_PST_PET[(df_PST_PET['PST'] <= time_for_getting_period_interval) & (df_PST_PET['PET'] > time_for_getting_period_interval)]

            PET_for_time = PST_and_PET_for_time['PET'].iloc[0]

            date_object = date(1900, 1, 1)
            PET_for_time = datetime.combine(date_object, PET_for_time)
            PET_for_time = (PET_for_time - timedelta(minutes = 1)).time()

            pet = PET_for_time  
            pst = PST_and_PET_for_time['PST'].iloc[0] 

            pet_datetime = datetime.combine(datetime.today(), pet)
            pst_datetime = datetime.combine(datetime.today(), pst)
            time_datetime_for_getting_period_interval = datetime.combine(datetime.today(), time_for_getting_period_interval)
            time_difference = pet_datetime - pst_datetime

            def y3(x1, x2, x3, y1, y2):
                return (y1 * (x2 - x3) + y2 * (x3 - x1)) / (x2 - x1)

            y = []
            x1 = 0
            x2 = time_difference.seconds + 60
            x3 = (time_datetime_for_getting_period_interval - pst_datetime).seconds

            for i, j in zip(time_point_runtimes_p_ordered, time_point_runtimes_n_ordered):
                y1 = time_point_runtimes_p_ordered[i]
                y2 = time_point_runtimes_n_ordered[j]
                y3_value = y3(x1, x2, x3, y1, y2)
                y.append(y3_value)

            y_new = []

            for i in y:
                integer_part = math.floor(i)
                fractional_part = i - integer_part
                if ((fractional_part >= 0.2500) and (fractional_part <= 0.7499)):
                    fractional_part = 0.5
                    i = integer_part + fractional_part
                elif ((fractional_part >= 0.7500) and (fractional_part <= 0.9999)):
                    integer_part = integer_part + 1
                    fractional_part = 0.0
                    i = integer_part + fractional_part
                elif ((fractional_part >= 0.0000) and (fractional_part <= 0.2499)):
                    fractional_part = 0.0
                    i = integer_part + fractional_part
                else:
                    i = i
                y_new.append(i)

            time_point_runtimes = {'IrvBwy - IrvAsh': 0, 'IrvAsh - IrvWes': 0, 'IrvWes - IrvKim': 0, 'IrvKim - IrvBlu': 0, 'IrvBlu - IrvCic': 0, 'IrvCic - IrvCen': 0, 'IrvCen - IrvHar': 0, 'IrvHar - IrvCum': 0}

            for i, j in zip(time_point_runtimes, range(len(y_new))):
                time_point_runtimes[i] = y_new[j]

            time_format = "%H:%M:%S"
            starting_time_for_cumulative_times = datetime.strptime(time, time_format)

            time_point_order = {'IrvAsh': 0, 'IrvWes': 0, 'IrvKim': 0, 'IrvBlu': 0, 'IrvCic': 0, 'IrvCen': 0, 'IrvHar': 0, 'IrvCum': 0}
            cumulative_times = {'IrvBwy': starting_time_for_cumulative_times, 'IrvAsh': 0, 'IrvWes': 0, 'IrvKim': 0, 'IrvBlu': 0, 'IrvCic': 0, 'IrvCen': 0, 'IrvHar': 0, 'IrvCum': 0}

            for i, j in zip(time_point_runtimes, time_point_order):
                time_point_order[j] = time_point_runtimes[i]

            time_point_order_list = list(time_point_order.values())     
            cumulative_times_list = list(cumulative_times.values()) 

            for i, j in zip(range(len(time_point_order_list)), range(len(cumulative_times_list))):
                cumulative_times_list[j+1] = cumulative_times_list[j] + timedelta(minutes = time_point_order_list[i]) 

            for i in range(len(cumulative_times_list)):
                cumulative_times_list[i] = cumulative_times_list[i].time()

            for key, i in zip(cumulative_times, range(len(cumulative_times_list))):
                cumulative_times[key] = cumulative_times_list[i]

        elif len(time_point_runtime) != 0:
            total_run_time = 0

            for i in time_point_runtime:
                total_run_time = total_run_time + time_point_runtime[i]

            time_point_runtimes = {'IrvBwy - IrvAsh': 0, 'IrvAsh - IrvWes': 0, 'IrvWes - IrvKim': 0, 'IrvKim - IrvBlu': 0, 'IrvBlu - IrvCic': 0, 'IrvCic - IrvCen': 0, 'IrvCen - IrvHar': 0, 'IrvHar - IrvCum': 0}

            for i in time_point_runtime:
                time_point_runtimes[i] = time_point_runtime[i]

            time_format = "%H:%M:%S"
            time = datetime.strptime(time, time_format)

            time_point_order = {'IrvAsh': 0, 'IrvWes': 0, 'IrvKim': 0, 'IrvBlu': 0, 'IrvCic': 0, 'IrvCen': 0, 'IrvHar': 0, 'IrvCum': 0}
            cumulative_times = {'IrvBwy': time, 'IrvAsh': 0, 'IrvWes': 0, 'IrvKim': 0, 'IrvBlu': 0, 'IrvCic': 0, 'IrvCen': 0, 'IrvHar': 0, 'IrvCum': 0}

            for i, j in zip(time_point_runtimes, time_point_order):
                time_point_order[j] = time_point_runtimes[i]

            time_point_order_list = list(time_point_order.values())     
            cumulative_times_list = list(cumulative_times.values()) 

            for i, j in zip(range(len(time_point_order_list)), range(len(cumulative_times_list))):
                cumulative_times_list[j+1] = cumulative_times_list[j] + timedelta(minutes = time_point_order_list[i]) 

            for i in range(len(cumulative_times_list)):
                cumulative_times_list[i] = cumulative_times_list[i].time()

            for key, i in zip(cumulative_times, range(len(cumulative_times_list))):
                cumulative_times[key] = cumulative_times_list[i]

        else:
            pass

        for i in cumulative_times:
            comparision_test_dataset.loc[len(comparision_test_dataset.index)] = [trip_no, i, cumulative_times[i]]

        trip_no = trip_no + 1
        test_start_time_datetime = test_start_time_datetime + timedelta(seconds = 30)
        time = test_start_time_datetime.strftime("%H:%M:%S")
        
    return comparision_test_dataset